## Libraries Used

In [337]:
from ftplib import FTP
import string
import os
import time
import math

import pprint
# prints the formatted representation of PrettyPrinter object
pp = pprint.PrettyPrinter(indent=4)


import boto3
# Get the service client 
client = boto3.client('s3')


## Setting FTP Connection

In [338]:
# from ftplib import FTP

# Domain name or server ip:
ftp = FTP('ec2-13-127-221-116.ap-south-1.compute.amazonaws.com')
    
ftp.login(user='test_user', passwd = 'rishabh')

'230 Logged on'

### Get all the file names

In [339]:
# Set the current directory on the server
ftp.cwd('/files/ftp')

# Return a list of file names
ftp_files = ftp.nlst()

# Printing
pp.pprint(ftp_files)

[   '9G9VYW23CU.csv',
    'Big-Data-Landscape-2017.pdf',
    'Big-Data-Landscape-2018.pdf',
    'Big-Data-Landscape-2019.pdf',
    'D029LRLIRA.csv',
    'DN7A49XY69.csv',
    'EDTMQD3VFB.csv',
    'file.txt',
    'GJNZEL7QS7.csv',
    'Measurement_info.csv',
    'Measurement_summary.csv',
    'MOCK_DATA.json',
    'N84NQPAZ5A.csv',
    'people.json',
    'SG75B3AMDD.csv',
    'WMM6GGSTIQ.csv']


### Get only CSV files 

In [340]:
def filter_names(n) :
    if n.endswith('csv') :
        return True
    else :
        return False

ftp_files = list(filter( filter_names, ftp_files ))

# Printing
pp.pprint(ftp_files)

[   '9G9VYW23CU.csv',
    'D029LRLIRA.csv',
    'DN7A49XY69.csv',
    'EDTMQD3VFB.csv',
    'GJNZEL7QS7.csv',
    'Measurement_info.csv',
    'Measurement_summary.csv',
    'N84NQPAZ5A.csv',
    'SG75B3AMDD.csv',
    'WMM6GGSTIQ.csv']


### Files which are already present in S3

In [341]:
try :
    # Returns some or all of the objects in a bucket
    s3_files = client.list_objects_v2(Bucket = 'rishabhsengar2611')
except :
    # if wrong bucket name is entered
    print("No such Bucket \n")

pp.pprint(s3_files)

{   'Contents': [   {   'ETag': '"a0ad38f3a09a0d0d0c2dd905736e6986"',
                        'Key': '9G9VYW23CU.csv',
                        'LastModified': datetime.datetime(2020, 4, 6, 6, 23, 58, tzinfo=tzutc()),
                        'Size': 354,
                        'StorageClass': 'STANDARD'},
                    {   'ETag': '"571d1690191e982850f6e593049ff428"',
                        'Key': 'D029LRLIRA.csv',
                        'LastModified': datetime.datetime(2020, 4, 6, 6, 23, 58, tzinfo=tzutc()),
                        'Size': 341,
                        'StorageClass': 'STANDARD'},
                    {   'ETag': '"8d2a79db85621494aa0c00583b0058cc"',
                        'Key': 'DN7A49XY69.csv',
                        'LastModified': datetime.datetime(2020, 4, 6, 6, 23, 58, tzinfo=tzutc()),
                        'Size': 467,
                        'StorageClass': 'STANDARD'},
                    {   'ETag': '"f903d3cf0c7f126fa9f939c3e07ee66d"',
         

In [342]:
# Dictionary Access to get the name of the first file
s3_files['Contents'][0]['Key']

'9G9VYW23CU.csv'

In [343]:
# Dictionary Access to get the size of the first file
s3_files['Contents'][0]['Size']

354

### Get S3 file names and there size

In [344]:
# Used to store the s3 files names
s3_files_names = []

# Used to store the sizes of files
s3_files_size = []

print("File Name \t\t Size\n")
# Print the files
try :
    for i in range( 0,len(s3_files['Contents']) ) :
        s3_files_names.append(s3_files['Contents'][i]['Key'])
        print(s3_files['Contents'][i]['Key'], end='\t\t')
        s3_files_size.append(s3_files['Contents'][i]['Size'])
        print(s3_files['Contents'][i]['Size'], end='\n')
        
except KeyError :
    print("Bucket is Empty")


File Name 		 Size

9G9VYW23CU.csv		354
D029LRLIRA.csv		341
DN7A49XY69.csv		467
EDTMQD3VFB.csv		237
GJNZEL7QS7.csv		316
N84NQPAZ5A.csv		429
SG75B3AMDD.csv		387
WMM6GGSTIQ.csv		277
chunk_1/		0
file.txt		319
people.json		1612897


### Remove already present file in the s3

In [345]:
# Already present files
s3_files_present = []

def filter_names(n) :
    if n in s3_files_names :
        s3_files_present.append(n)
        return False
    else :
        return True

ftp_files = list(filter( filter_names, ftp_files ))


print(" To be Added")
pp.pprint(ftp_files)
print("\n Already Present")
pp.pprint(s3_files_present)

 To be Added
['Measurement_info.csv', 'Measurement_summary.csv']

 Already Present
[   '9G9VYW23CU.csv',
    'D029LRLIRA.csv',
    'DN7A49XY69.csv',
    'EDTMQD3VFB.csv',
    'GJNZEL7QS7.csv',
    'N84NQPAZ5A.csv',
    'SG75B3AMDD.csv',
    'WMM6GGSTIQ.csv']


In [346]:
# Make the Dictionary of filename and Size of the files present in FTP 
ftp_size = {}
for i in ftp_files :
# FTP.size(filename) ---- >Request the size of the file named filename on the server.
    ftp_size[i] = ftp.size(i)

pp.pprint(ftp_size)

{'Measurement_info.csv': 124452984, 'Measurement_summary.csv': 94076158}


## Adding files to S3

In [347]:
# Each part must be at least 5 MB in size
# Since AWS won't allow us to have size less than 5MB
# 1024*1024*6 == 6MB
chunk_size = 6291456

In [348]:
# ftp.size(ftp_files[0])
for file in ftp_size :
    print(file+"\t\t"+str(ftp_size[file]))

Measurement_info.csv		124452984
Measurement_summary.csv		94076158


### Files which have size less than 6MB uploaded

In [349]:
# Used to store the large files
big_files = []

# This for loop is used to upload files
for file in ftp_size :
    if ftp_size[file] < chunk_size :
        
        # Change the location
        local_file = os.path.join('/home/bluepi/Desktop/',file)
        
        # Retrieve a file in binary transfer mode
        # RETR command --> A RETR request asks the server to send the contents of a file 
        #                       over the data connection already established by the client.
        ftp.retrbinary('RETR ' + file, open(local_file, 'wb').write)
        print("retieved file :\t" + file,end='\n' )
        
        # Upload file in binary mode in s3 object
        # Useful when we perform multipsrt upload
        with open(local_file, 'rb') as data:
            client.upload_fileobj(Fileobj=data, Bucket= 'rishabhsengar2611', Key= file)
            
        print("uploaded file :\t" + file,end='\n')
        #os.remove(file)
    else :
        big_files.append(file)
        print("big files :  "+file+"  -->  size :  "+str(ftp_size[file]),end = '\n')
        
# Errors :-
# Brokenpipeerror errno 32
# ftplib.error_perm: 530 Login authentication failed

big files :  Measurement_info.csv  -->  size :  124452984
big files :  Measurement_summary.csv  -->  size :  94076158


### Files to be uploaded in chunks( Working on it )

In [350]:
chunk_size

6291456

In [351]:
chunk_count = math.ceil(ftp_size[big_files[0]] / chunk_size)
multipart_upload =client.create_multipart_upload(Bucket = 'rishabhsengar2611',Key = 'Measurement_summary.csv')




In [352]:
print(type(multipart_upload['UploadId']))

<class 'str'>


In [353]:
# response = client.upload_part(
#     Body=file,
#     Bucket='rishabhsengar2611',
#     Key='Measurement_summary.csv',
#     PartNumber='1',
#     UploadId=multipart_upload['UploadId'])

In [354]:
# This will handle the data being downloaded
# It will be explained shortly
i=0
j=0
l = []
def handleDownload(block):
    global i,j
    global l
    if os.stat('Measurement_summary.csv').st_size < chunk_size :
        #file.write(block)
        l.append(block)
        print(j+1)
        #print(os.stat('Measurement_summary.csv').st_size,end='\t')
    else :
        print("Uploading----")
        
        response = client.upload_part(
        Body=b'l',
        Bucket='rishabhsengar2611',
        Key='Measurement_summary.csv',
        PartNumber=i+1,
        UploadId=multipart_upload['UploadId'])
        l = []
        print("\t Uploaded \n")
    
# Open the file for writing in binary mode
print('Opening local file ' + big_files[1])
file = open(big_files[1], 'wb')

# Download the file a chunk at a time
# Each chunk is sent to handleDownload
# We append the chunk to the file and then print a '.' for progress
# RETR is an FTP command
print('Getting ' + big_files[1])
ftp.retrbinary('RETR ' + big_files[1],handleDownload,blocksize=chunk_size)

Opening local file Measurement_summary.csv
Getting Measurement_summary.csv
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


limit_output extension: Maximum message size of 10000 exceeded with 10105 characters

# Lambda Code "Same as above"

In [ ]:
from ftplib import FTP
import string
import pprint
import time
import boto3
import os
client = boto3.client('s3')


def lambda_handler(event, context):
    
    try:

        ftp = FTP('13.233.66.93')
        ftp.login(user='test_user', passwd = 'rishabh')
        ftp.cwd('/files/ftp')
        ftp_files = ftp.nlst()
        #print(ftp_files,end='\n')
        
        
        def filter_names(n) :
            if n.endswith('csv') :
                return True
            else :
                return False

        ftp_files = list(filter( filter_names, ftp_files ))
        #print(ftp_files)
   
    except:
        print("Error connecting to FTP")
    
    try :
        s3_files = client.list_objects_v2(Bucket = 'rishabhsengar2611')
    except :
        print("No such Bucket \n")

# Returns some or all (up to 1,000) of the objects in a bucket
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(s3_files)

    
    
    s3_files_names = []
    s3_files_size = []
    #print("File Name \t Size\n")
    # Print the files
    try :
        for i in range( 0,len(s3_files['Contents']) ) :
            s3_files_names.append(s3_files['Contents'][i]['Key'])
            #print(s3_files['Contents'][i]['Key'], end='\t')
            s3_files_size.append(s3_files['Contents'][i]['Size'])
            #print(s3_files['Contents'][i]['Size'], end='\n')
        
    except KeyError :
        print("Bucket is Empty")


    
    def filter_names(n) :
        if n in s3_files_names :
            return False
        else :
            return True

    ftp_files = list(filter( filter_names, ftp_files ))
    #print(ftp_files)

    
    
    ftp_size = {}
    for i in ftp_files :
        ftp_size[i] = ftp.size(i)
    #print(ftp_size)



    chunk_size = 6291456
    
    
    
    big_files = []
    for file in ftp_size :
        if ftp_size[file] < chunk_size :
            localfile = os.path.join('/tmp/', file)
            
            ftp.retrbinary('RETR ' + file, open( localfile, 'wb').write)
            print("retieved file :\t" + file,end='\n' )
            
            with open(localfile, 'rb') as data:
                client.upload_fileobj(data, 'rishabhsengar2611', file)
            print("uploaded file :\t" + file,end='\n')
            #os.remove(file)
            
        else :
            big_files.append(file)
            print("big files :  "+file+"  -->  size :  "+str(ftp_size[file]),end = '\n')

    return "Success"
